<a href="https://colab.research.google.com/github/Jignacio14/7506R-1C2023-GRUPO09/blob/Pruebas/7506R_TP1_GRUPO9_CHP1_ENTREGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pruebas

En este apartado se estara trabajando el analisis exploratorio de los datos que se estaran usando para el trabajo practico #1

In [1]:
import pandas as pd 
import numpy as np
import sklearn as sk
import seaborn as sns
import matplotlib as mpl

In [2]:
hotelsDfOriginal = pd.read_csv("./hotels_train.csv")
hotelsdf = hotelsDfOriginal.copy()

print("El data frame esta compuesto por "f"{hotelsdf.shape[0]}"" filas y "f"{hotelsdf.shape[1]}"" columnas")

El data frame esta compuesto por 61913 filas y 33 columnas


Un vistaso básico a la información contenida en el dataframe viene dada por:

In [3]:
pd.concat([hotelsdf.head(2), hotelsdf.sample(5), hotelsdf.tail(2)])

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,id,is_canceled
0,City Hotel,49,2016,September,37,5,1,2,1,0.0,...,NaN,0,Transient,115.50,0,1,Canceled,2016-08-25,7aa4cc6b-b92c-4061-b21d-4f9e0cac4689,1
1,Resort Hotel,4,2015,October,44,31,0,1,2,0.0,...,NaN,0,Transient,42.00,1,0,Check-Out,2015-11-01,b1428f80-c56c-4ae4-91a9-6962edae08b1,0
48201,City Hotel,328,2015,September,37,10,0,2,2,0.0,...,NaN,0,Contract,62.00,0,0,Canceled,2015-01-01,5a00818a-607b-4ca7-9c6a-4bdacbd8122a,1
56645,City Hotel,164,2016,July,31,27,2,5,2,0.0,...,NaN,0,Transient,99.45,0,0,Canceled,2016-02-25,3e462e48-8ebc-4e59-a15f-c37686d52f0f,1
20934,City Hotel,0,2017,January,2,11,0,1,2,0.0,...,NaN,0,Transient,90.00,0,0,Check-Out,2017-01-12,933c5ef3-30b9-413a-b8d0-e390afea7348,0
6062,Resort Hotel,13,2016,February,6,5,0,2,2,0.0,...,NaN,0,Transient,58.00,0,0,Check-Out,2016-02-07,5f1ca4e2-884e-4cab-b2b6-e334109b03fe,0
48600,Resort Hotel,96,2016,June,24,8,0,4,1,0.0,...,NaN,0,Transient,107.00,0,0,Check-Out,2016-06-12,cbd88eb3-784d-43f4-b64c-a1d23865bb09,0
61911,Resort Hotel,143,2015,October,41,10,2,5,2,0.0,...,NaN,0,Transient,36.05,0,0,Canceled,2015-07-08,01906859-1094-4e37-b13e-b015b594d159,1
61912,City Hotel,57,2016,April,17,17,2,5,2,0.0,...,NaN,0,Transient,143.29,0,0,Canceled,2016-04-09,7137e599-4442-41fb-b147-4aba9f344e38,1


In [4]:
hotelsdf.rename(columns = {
    'hotel':'hotel',
    'lead_time':'anticipacion_dias',
    'arrival_date_year':'llegada_anio',
    'arrival_date_month':'llegada_mes',
    'arrival_date_week_number':'llegada_numero_semana',
    'arrival_date_day_of_month':'llegada_dia_del_mes',
    'stays_in_weekend_nights':'cantidad_noches_finDeSemana',
    'stays_in_week_nights':'cantidad_noches_semana',
    'adults':'cantidad_adultos',
    'children':'cantidad_ninios', #En el paper, dice que children es un int. Pero figura como float64. #TODO: Investigar
    'company':'ID_compania',
    'days_in_waiting_list': 'cantidad_dias_listaEspera',
    'customer_type': 'tipo_cliente',
    'adr': 'promedio_gastos_diarios', #No me termina de cerrar este valor
    'required_car_parking_spaces': 'cantidad_estacionamientos_pedidos',
    'total_of_special_requests': 'cantidad_pedidos_especiales',
    'reservation_status': 'estado_reserva',
    'reservation_status_date': 'estado_reserva_fecha',
    'id': 'id',
    'is_canceled': 'esta_cancelado',


    
}, inplace = True)

In [5]:
pd.concat([hotelsdf.head(2), hotelsdf.sample(5), hotelsdf.tail(2)])

,hotel,anticipacion_dias,llegada_anio,llegada_mes,llegada_numero_semana,llegada_dia_del_mes,cantidad_noches_finDeSemana,cantidad_noches_semana,cantidad_adultos,cantidad_ninios,...,ID_compania,cantidad_dias_listaEspera,tipo_cliente,promedio_gastos_diarios,cantidad_estacionamientos_pedidos,cantidad_pedidos_especiales,estado_reserva,estado_reserva_fecha,id,esta_cancelado
0,City Hotel,49,2016,September,37,5,1,2,1,0.0,...,NaN,0,Transient,115.50,0,1,Canceled,2016-08-25,7aa4cc6b-b92c-4061-b21d-4f9e0cac4689,1
1,Resort Hotel,4,2015,October,44,31,0,1,2,0.0,...,NaN,0,Transient,42.00,1,0,Check-Out,2015-11-01,b1428f80-c56c-4ae4-91a9-6962edae08b1,0
2896,Resort Hotel,237,2016,August,32,1,1,4,2,0.0,...,NaN,0,Transient,126.00,0,0,Canceled,2015-12-10,edd4b904-7e4a-4e2b-8bc0-a9f43bdfd133,1
60052,Resort Hotel,154,2016,July,30,23,2,2,3,0.0,...,NaN,0,Transient,115.00,0,2,Check-Out,2016-07-27,b1753e4e-d0ec-4443-862d-d75b8e30e887,0
59874,Resort Hotel,112,2017,March,11,15,0,3,1,0.0,...,477.0,0,Transient,40.00,0,0,Check-Out,2017-03-18,2bd4db79-0cb2-470f-b7c9-11620b8f53c0,0
18115,City Hotel,100,2015,October,42,16,0,1,2,0.0,...,NaN,0,Transient,108.00,0,0,Canceled,2015-09-28,516e1fa3-8754-4eda-b6f0-4af2ff16530d,1
9610,Resort Hotel,141,2017,June,23,6,0,3,2,0.0,...,NaN,0,Transient,126.00,0,2,Check-Out,2017-06-09,1b0a8c7e-fb4b-41f9-96a4-2ab2ab974419,0
61911,Resort Hotel,143,2015,October,41,10,2,5,2,0.0,...,NaN,0,Transient,36.05,0,0,Canceled,2015-07-08,01906859-1094-4e37-b13e-b015b594d159,1
61912,City Hotel,57,2016,April,17,17,2,5,2,0.0,...,NaN,0,Transient,143.29,0,0,Canceled,2016-04-09,7137e599-4442-41fb-b147-4aba9f344e38,1


Por otro lado un vistazo un poco más detallado a la informacion contenida por el dataframe puede verse:

In [6]:
hotelsdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61913 entries, 0 to 61912
Data columns (total 33 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   hotel                              61913 non-null  object 
 1   anticipacion_dias                  61913 non-null  int64  
 2   llegada_anio                       61913 non-null  int64  
 3   llegada_mes                        61913 non-null  object 
 4   llegada_numero_semana              61913 non-null  int64  
 5   llegada_dia_del_mes                61913 non-null  int64  
 6   cantidad_noches_finDeSemana        61913 non-null  int64  
 7   cantidad_noches_semana             61913 non-null  int64  
 8   cantidad_adultos                   61913 non-null  int64  
 9   cantidad_ninios                    61909 non-null  float64
 10  babies                             61913 non-null  int64  
 11  meal                               61913 non-null  obj

Segun la clasificacion de las variables podemos establecer lo siguiente sobre los datos extraidos del dataframe

Variables cuantitativas, entre las cuales podemos encontrar:

· arrival_date_year

· arrival_date_month 

· arrival_date_week_number  
  
· arrival_date_day_of_month

· stays_in_weekend_nights

· days_in_waiting_list

· stays_in_week_nights

· required_car_parking_spaces

continuar...

In [7]:
hotelsdf.isnull().sum()

hotel                                    0
anticipacion_dias                        0
llegada_anio                             0
llegada_mes                              0
llegada_numero_semana                    0
llegada_dia_del_mes                      0
cantidad_noches_finDeSemana              0
cantidad_noches_semana                   0
cantidad_adultos                         0
cantidad_ninios                          4
babies                                   0
meal                                     0
country                                221
market_segment                           0
distribution_channel                     0
is_repeated_guest                        0
previous_cancellations                   0
previous_bookings_not_canceled           0
reserved_room_type                       0
assigned_room_type                       0
booking_changes                          0
deposit_type                             0
agent                                 7890
ID_compania

In [8]:
hotelsdf['hotel'].value_counts().index.tolist()

['City Hotel', 'Resort Hotel']